In [21]:
import numpy as np

import tensorflow as tf
from tensorflow import keras

In [22]:
#!wget https://github.com/alexeygrigorev/large-datasets/releases/download/dogs-cats-model/dogs_cats_10_0.687.h5 -O dogs_cats_model.h5

In [23]:
model = keras.models.load_model('xception_v1_05_0.975.h5')

In [24]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

def preprocess_input(x):
    x /= 255
    return x

In [25]:
url = 'https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg'
image_size = (256, 256)

img = download_image(url)
img = prepare_image(img, target_size=image_size)

In [26]:
x = np.array(img, dtype='float32')
X = np.array([x])

X = preprocess_input(X)
X

array([[[[0.6666667 , 0.73333335, 0.70980394],
         [0.5686275 , 0.63529414, 0.6117647 ],
         [0.52156866, 0.5882353 , 0.5647059 ],
         ...,
         [0.6901961 , 0.7490196 , 0.7607843 ],
         [0.7254902 , 0.8       , 0.8156863 ],
         [0.7058824 , 0.8       , 0.80784315]],

        [[0.6392157 , 0.7058824 , 0.68235296],
         [0.6313726 , 0.69803923, 0.6745098 ],
         [0.69411767, 0.7607843 , 0.7372549 ],
         ...,
         [0.5647059 , 0.6313726 , 0.60784316],
         [0.7137255 , 0.7921569 , 0.78431374],
         [0.6431373 , 0.7411765 , 0.7294118 ]],

        [[0.57254905, 0.6392157 , 0.6156863 ],
         [0.6431373 , 0.70980394, 0.6862745 ],
         [0.62352943, 0.6901961 , 0.6666667 ],
         ...,
         [0.6313726 , 0.7019608 , 0.6392157 ],
         [0.6       , 0.6862745 , 0.6392157 ],
         [0.7882353 , 0.8784314 , 0.85490197]],

        ...,

        [[0.49019608, 0.53333336, 0.47843137],
         [0.42352942, 0.4627451 , 0.41960785]

In [27]:
pred = model.predict(X)

In [28]:
classes = [
    'butterfly',
    'cats',
    'chicken',
    'cow',
    'dog',
    'elephant',
    'horse',
    'sheep',
    'spider',
    'squirrel'    
]

dict(zip(classes, pred[0].round(6)))

{'butterfly': 1e-06,
 'cats': 5e-06,
 'chicken': 5e-06,
 'cow': 5.3e-05,
 'dog': 0.999787,
 'elephant': 4e-06,
 'horse': 2e-06,
 'sheep': 0.000109,
 'spider': 3.2e-05,
 'squirrel': 1e-06}

In [29]:
## Convert Keras to TF-Lite

In [30]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

with open('xception_v1.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/wk/yt_rdhdx7mqbrqjqngjkkqx00000gn/T/tmp1qq2k09m/assets


INFO:tensorflow:Assets written to: /var/folders/wk/yt_rdhdx7mqbrqjqngjkkqx00000gn/T/tmp1qq2k09m/assets


In [31]:
#!pip install --extra-index-url https://google-coral.github.io/py-repo/ tflite_runtime

In [32]:
#!pip install tflite_runtime-2.5.0-cp38-cp38-macosx_10_15_x86_64.whl

In [33]:
import tensorflow.lite as tflite
#import tflite_runtime.interpreter as tflite

In [34]:
interpreter = tflite.Interpreter(model_path='xception_v1.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

In [35]:
output_index

228

In [36]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
pred = interpreter.get_tensor(output_index)

In [37]:
classes = [
    'butterfly',
    'cats',
    'chicken',
    'cow',
    'dog',
    'elephant',
    'horse',
    'sheep',
    'spider',
    'squirrel'    
]

dict(zip(classes, pred[0].round(6)))

{'butterfly': 1e-06,
 'cats': 4e-06,
 'chicken': 5e-06,
 'cow': 5.5e-05,
 'dog': 0.999837,
 'elephant': 3e-06,
 'horse': 1e-06,
 'sheep': 6.2e-05,
 'spider': 3.2e-05,
 'squirrel': 1e-06}